# CREST Prediction Control

The sheet takes through the control pipeline. Data is retrieved through API and feed into the offline AI models. The AI models predict the temperatures, CO2 and TVOCs in the space and feed into the weighted average algorithm which then feeds the input temperature and iaq into fuzzy logic controller. The output from the controller is sent to the Niagara 4 BMS platform through the MQTT broker. (Some parts of the codes are removed due to IP protection)

In [ ]:
#Import relevant library

import requests
import pandas as pd
import json
import numpy as np
import pickle
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import math
import paho.mqtt.client as mqtt
import time,sys
import logging

In [ ]:
import os
os.getcwd()

In [ ]:
#change working directory"
os.chdir("D:\\OneDrive - Loughborough University\\PhD\\Case study\\Crest\\data_ai_model")

## API

In [ ]:
#API to retrive the data

url = 'https://well.arbnco.com/api/v1'
api = '{url}/sites/{site_id}/devices/{device_id}/readings'.format(url = url, site_id = 311, device_id = 3918)

In [ ]:
api_key = "****************************"
# the api-key below is limited to 200 queries per day.
headers = {"accept": "application/json", 'Authorization': api_key}

## AI Models

In [ ]:
# load model
with open('xgb_temp_aq1.pkl', 'rb') as f:
    reg_temp_aq1 = pickle.load(f)
with open('xgb_temp_aq2.pkl', 'rb') as f:
    reg_temp_aq2 = pickle.load(f)
with open('xgb_temp_aq3.pkl', 'rb') as f:
    reg_temp_aq3 = pickle.load(f)
with open('xgb_co2_aq1.pkl', 'rb') as f:
    reg_co2_aq1 = pickle.load(f)
with open('xgb_co2_aq2.pkl', 'rb') as f:
    reg_co2_aq2 = pickle.load(f)
with open('xgb_co2_aq3.pkl', 'rb') as f:
    reg_co2_aq3 = pickle.load(f)
with open('xgb_tvoc_aq1.pkl', 'rb') as f:
    reg_tvoc_aq1 = pickle.load(f)
with open('xgb_tvoc_aq2.pkl', 'rb') as f:
    reg_tvoc_aq2 = pickle.load(f)
with open('xgb_tvoc_aq3.pkl', 'rb') as f:
    reg_tvoc_aq3 = pickle.load(f)

## MQTT setup

In [ ]:
topic_crest="crest/#"
topic_crest2="crest/+/+"

topic1="crest/Common/Josh_Control_Active"
topic2="crest/vav_gj08/settemp"
topic3="crest/vav_gj09/settemp"
topic4="crest/vav_gj08/flowrate"
topic5="crest/vav_gj09/flowrate"
topic6="crest/Common/josh_spacetemp_08"
topic7="crest/Common/josh_spacetemp_09"

sub_topic=topic_crest2

#sub_topic=topic_lights
logging.basicConfig(level=logging.INFO)
#use DEBUG,INFO,WARNING

def on_log(client, userdata, level, buf):
    logging.info(buf) 
def on_connect(client, userdata, flags, rc):
    if rc==0:
        client.connected_flag=True #set flag
        logging.info("connected OK")
    else:
        logging.info("Bad connection Returned code="+str(rc))
        client.loop_stop()  
def on_disconnect(client, userdata, rc):
    logging.info("client disconnected ok")
def on_publish(client, userdata, mid):
    logging.info("In on_pub callback mid= "  + str(mid))
def on_subscribe(client, userdata, mid, granted_qos):
    logging.info("subscribed MID= " + str(mid))
def on_message(client, userdata, message):
    topic=message.topic
    msgr=str(message.payload.decode("utf-8"))
    msgr="Message Received topic="+topic+ " message ="+msgr
    logging.info(msgr)


Create a scheduler which schedules pings the api and runs the code every xx minutes

In [ ]:
from datetime import datetime, timedelta
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger
from apscheduler.events import EVENT_JOB_ERROR, EVENT_JOB_EXECUTED

# Fuzzy logic libraries

from fuzzy_expert.variable import FuzzyVariable
from fuzzy_expert.rule import FuzzyRule
from fuzzy_expert.inference import DecompositionalInference

In [ ]:
def api_call_control():
    #Extracting timestamp
    initial_time = datetime.now() - timedelta(hours = 0.5)
    from_ = str(initial_time.timestamp())
    to_ = str(datetime.now().timestamp())
    
    #Setting up paramters to load from API
    payload = {
    'request_cache' : 'true',
    'grouped_by_devices' : 'true',
    'parameter' : 'temperature',
    'parameter' : 'co2',
    'parameter' : 'tvoc',
    'from': from_,
    'to': to_ 
    }
    
    response = requests.get(api , params=payload, headers=headers)
    
    #Extract data from json file
    raw_data = json.loads(response.content.decode('utf8').replace("'", '"'))
    
    sensor_aq4_list = []
    for x in raw_data['data']:
        sensor_aq4_list.append(x)
        
    #If there is no data at the time.    
    assert (len(sensor_aq4_list) > 0), "Empty Data"
    
    print('Data', raw_data['data'][sensor_aq4_list[0]]['3918']['temperature'],raw_data['data'][sensor_aq4_list[0]]['3918']['co2'],raw_data['data'][sensor_aq4_list[0]]['3918']['tvoc'], sensor_aq4_list[0])
    
    t = sensor_aq4_list[0]
    
    sensor_aq4_temp = float(raw_data['data'][sensor_aq4_list[0]]['3918']['temperature'])
    sensor_aq4_co2 = float(raw_data['data'][sensor_aq4_list[0]]['3918']['co2'])
    sensor_aq4_tvoc = float(raw_data['data'][sensor_aq4_list[0]]['3918']['tvoc'])
    
    b = datetime.strptime(t[:-4], "%Y-%m-%d %H:%M:%S")
    
    temp_x = []
    
    temp_x.append(sensor_aq4_temp)
    temp_x.append(*******)
    temp_x.append(*******)
    temp_x.append(*******)
    temp_x.append(*******)
    
    co2_x = []
    
    co2_x.append(sensor_aq4_co2)
    co2_x.append(********)
    co2_x.append(********)
    co2_x.append(********)
    co2_x.append(********)
    
    tvoc_x = []
    
    tvoc_x.append(sensor_aq4_tvoc)
    tvoc_x.append(*******)
    tvoc_x.append(*******)
    tvoc_x.append(*******)
    tvoc_x.append(*******)    
    #inputs from aq4 sensor for prediction
    
    temp_x_ = np.array(temp_x).reshape(1,5)
    co2_x_ = np.array(co2_x).reshape(1,5)
    tvoc_x_ = np.array(tvoc_x).reshape(1,5)
    
    #predicted temperature for aq1 sensor
    temp_aq1_predict = reg_temp_aq1.predict(temp_x_)[0]
    temp_aq2_predict = reg_temp_aq2.predict(temp_x_)[0]
    temp_aq3_predict = reg_temp_aq3.predict(temp_x_)[0]
    
    #predicted temperature for aq2 sensor
    co2_aq1_predict = reg_co2_aq1.predict(co2_x_)[0]
    co2_aq2_predict = reg_co2_aq2.predict(co2_x_)[0]
    co2_aq3_predict = reg_co2_aq3.predict(co2_x_)[0]
 
    #predicted temperature for aq3 sensor
    tvoc_aq1_predict = reg_tvoc_aq1.predict(tvoc_x_)[0]
    tvoc_aq2_predict = reg_tvoc_aq2.predict(tvoc_x_)[0]
    tvoc_aq3_predict = reg_tvoc_aq3.predict(tvoc_x_)[0]

    temp_aq_1 = np.round(temp_aq1_predict,1)
    temp_aq_2 = np.round(temp_aq2_predict,1)
    temp_aq_3 = np.round(temp_aq3_predict,1)
    temp_aq_4 = sensor_aq4_temp
  
    co2_aq_1 = np.round(co2_aq1_predict,1)
    co2_aq_2 = np.round(co2_aq2_predict,1)
    co2_aq_3 = np.round(co2_aq3_predict,1)
    co2_aq_4 = sensor_aq4_co2
     
    tvoc_aq_1 = np.round(tvoc_aq1_predict,1)
    tvoc_aq_2 = np.round(tvoc_aq2_predict,1)
    tvoc_aq_3 = np.round(tvoc_aq3_predict,1)
    tvoc_aq_4 = sensor_aq4_tvoc
    
    print("temp_aq1 -", temp_aq_1,"temp_aq2 -", temp_aq_2,"temp_aq3 -", temp_aq_3,"co2_aq1 -", co2_aq_1,"co2_aq2 -", co2_aq_2,"co2_aq3 -", co2_aq_3,"tvoc_aq1 -", tvoc_aq_1,"tvoc_aq2 -", tvoc_aq_2,"tvoc_aq3 -", tvoc_aq_3)
      
    #calculate the IAQ index
    #********************
    #********************
    #********************

    w_iaq = #***********
    
    #calculate the weighted average of temperature
    #**********************
    #*********************

    s1_r = round(s1,1)
    s2_r = round(s2,1)
    print("input_temp_vav8 - ", s1_r,"input_temp_vav9 - ", s2_r)
    
    #define inputs for fuzzy logic controller
    iaq_index = w_iaq
    input_temp_08 = s1_r
    input_temp_09 = s2_r

    
    #Define fuzzy control variables and their membership functions
    #********************
    #*******************
    
    #Write the fuzzy rules


    model = DecompositionalInference(
            and_operator="min",
            or_operator="max",
            implication_operator="Rc",
            composition_operator="max-min",
            production_link="max",
            defuzzification_operator="cog",
    )

    fuzzy_output = model(
            variables=variables,
            rules=rules,
            input_temp_08=input_temp_08,
            input_temp_09=input_temp_09,
            iaq_index=iaq_index,
    )

    # this lines UNPACKS values
    # of variable a
    (fuzzy, output) = fuzzy_output
    #for key,val in fuzzy.items():
            #exec(key + '=val')
    
    fan_speed_08 = fuzzy['fan_speed_08']
    fan_speed_09 = fuzzy['fan_speed_09']
    
    fan_speed_08_out = round(fan_speed_08)
    fan_speed_09_out = round(fan_speed_09)
    print(fuzzy)

    print("fan_speed_08 - ", fan_speed_08_out,"fan_speed_09 - ", fan_speed_09_out)
    
    mqtt.Client.connected_flag=False#create flag in class
    client = mqtt.Client("python1")             #create new instance 
    client.on_log=on_log
    client.on_connect = on_connect
    client.on_disconnect = on_disconnect
    client.on_publish = on_publish
    client.on_subscribe = on_subscribe
    client.on_message = on_message
    print("Connecting to broker ")
    try:
        client.connect("158.125.161.166", 1883, 60)      #connect to broker
    except:
        print("can't connect")
        sys.exit(1)
    print("subscribing to topic ",sub_topic)
    ret= client.subscribe(sub_topic,0)
    print("subscribed returned ",ret)
    #time.sleep(4)
    msg= "true"
    print("Publishing topic= ",topic1," message= ",msg)
    client.publish(topic1,msg)
    #time.sleep(4)
    msg= 20
    print("Publishing topic= ",topic2," message= ",msg)
    client.publish(topic2,msg)
    #time.sleep(4)
    msg= 20
    print("Publishing topic= ",topic3," message= ",msg)
    client.publish(topic3,msg)
    #time.sleep(4)
    msg= fan_speed_08_out
    print("Publishing topic= ",topic4," message= ",msg)
    client.publish(topic4,msg)
    #time.sleep(4)
    msg= fan_speed_09_out
    print("Publishing topic= ",topic5," message= ",msg)
    client.publish(topic5,msg)
    #time.sleep(4)
    msg= input_temp_08
    print("Publishing topic= ",topic6," message= ",msg)
    client.publish(topic6,msg)
    #time.sleep(4)
    msg= input_temp_09
    print("Publishing topic= ",topic7," message= ",msg)
    client.publish(topic7,msg)
    #time.sleep(4)

In [ ]:
#The scheduler schedules to run the above code every 15 minutes
scheduler = BackgroundScheduler()
interval = IntervalTrigger(minutes=15)

In [ ]:
#scheduler.add_listener(listener, EVENT_JOB_EXECUTED | EVENT_JOB_ERROR)
scheduler.add_job(api_call_control, interval, id="crest")

In [ ]:
scheduler.start()

In [ ]:
scheduler.shutdown()

In [ ]:
scheduler.remove_all_jobs()